In [1]:
def solve_cryptarithmetic(puzzle):
    words = puzzle[:-1].split('+')
    result_word = puzzle[-1]
    chars = set(''.join(words) + result_word)

    def is_valid_assignment(assignment):
        return sum(assignment[word[0]] for word in words) == assignment[result_word[0]]

    def is_consistent(char, digit, assignment):
        return char not in assignment or assignment[char] == digit

    def propagate_constraints(assignment):
        updated = True
        while updated:
            updated = False
            for word in words:
                if len(word) == 1:  # Si la palabra tiene solo un carácter, no se pueden propagar restricciones
                    continue
                if all(c in assignment for c in word[:-1]):
                    last_char = word[-1]
                    if last_char not in assignment:
                        possible_digits = [int(d) for d in range(10) if d not in assignment.values()]
                        consistent_digits = [d for d in possible_digits if is_consistent(last_char, d, assignment)]
                        if len(consistent_digits) == 1:
                            assignment[last_char] = consistent_digits[0]
                            updated = True

    def select_next_char(assignment):
        unassigned_chars = chars - assignment.keys()
        unassigned_chars = sorted(unassigned_chars, key=lambda char: calculate_degree(char), reverse=True)
        return unassigned_chars[0] if unassigned_chars else None

    def calculate_degree(char):
        return sum(1 for word in words if char in word)

    def backtrack_search(assignment):
        if len(assignment) == len(chars):
            if is_valid_assignment(assignment):
                return assignment
            else:
                return None

        char = select_next_char(assignment)
        if char is None:
            return None

        for digit in range(10):
            if is_consistent(char, digit, assignment):
                assignment[char] = digit
                propagate_constraints(assignment)  # Propagar restricciones después de cada asignación
                result = backtrack_search(assignment)
                if result:
                    return result
                assignment.pop(char)
        return None

    solution = backtrack_search({})
    return solution

# Example usage
if __name__ == "__main__":
    puzzle = "SEND+MORE=MONEY"
    solution = solve_cryptarithmetic(puzzle)
    if solution:
        print("Solución encontrada:")
        for char, digit in solution.items():
            print(f"{char}: {digit}")
    else:
        print("No se encontró solución.")

Solución encontrada:
N: 0
E: 0
D: 0
O: 0
R: 0
S: 0
=: 0
M: 0
Y: 0
